## Importing Dependencies

In [1]:
import os
import ast
import json
import random

import numpy as np
import pandas as pd

import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
from matplotlib.patches import Patch

## Loading the logs

In [2]:
# ====== CONFIG ======
log_file = "./logs/logs.txt"        # path to your log.txt
save_dir = "./enhanced_rt_detr_metric_plots"          # folder to save plots
os.makedirs(save_dir, exist_ok=True)

In [3]:
# ====== LOAD LOG DATA ======
records = []
with open(log_file, "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith("{"):
            records.append(json.loads(line))

print(f"Loaded {len(records)} epochs from log file")

Loaded 51 epochs from log file


In [4]:
# ====== ORGANIZE DATA ======
epochs = [r["epoch"] for r in records]

In [5]:
# Training losses
train_metrics = {}
for key in records[0]:
    if key.startswith("train_"):
        train_metrics[key] = [r[key] for r in records]

In [6]:
# Validation metrics (test_coco_eval_bbox has 12 entries)
coco_labels = [
    "mAP@[0.50:0.95]",
    "mAP@0.50",
    "mAP@0.75",
    "mAP small",
    "mAP medium",
    "mAP large",
    "AR@1",
    "AR@10",
    "AR@100",
    "AR small",
    "AR medium",
    "AR large"
]

coco_metrics = {label: [] for label in coco_labels}
for r in records:
    vals = r.get("test_coco_eval_bbox", [None]*12)
    for i, label in enumerate(coco_labels):
        coco_metrics[label].append(vals[i])

In [7]:
# ====== CREATE SINGLE FIGURE ======
fig, axs = plt.subplots(3, 3, figsize=(18, 15))
axs = axs.ravel()

# 1. Training total loss
axs[0].plot(epochs, train_metrics["train_loss"], label="Total Loss")
axs[0].set_title("Training Loss Curve")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend()
axs[0].grid(True, linestyle="--", alpha=0.6)

# 2. Training loss components
for k in ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]:
    axs[1].plot(epochs, train_metrics[k], label=k.replace("train_", ""))
axs[1].set_title("Training Loss Components")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Loss")
axs[1].legend()
axs[1].grid(True, linestyle="--", alpha=0.6)

# 3. Training DN losses
for k, v in train_metrics.items():
    if "dn" in k:
        axs[2].plot(epochs, v, label=k.replace("train_", ""))
axs[2].set_title("Training DN Losses")
axs[2].set_xlabel("Epoch")
axs[2].set_ylabel("Loss")
axs[2].legend(fontsize=8)
axs[2].grid(True, linestyle="--", alpha=0.6)

# 4. Training AUX losses (with smoothing for spikes)
start_epoch, end_epoch = 35, 40
for k, v in train_metrics.items():
    if "aux" in k:
        v = np.array(v)
        smoothed = v.copy()
        start_idx = next(i for i, e in enumerate(epochs) if e >= start_epoch)
        end_idx = next(i for i, e in enumerate(epochs) if e >= end_epoch)
        x0, x1 = epochs[start_idx - 1], epochs[end_idx + 1]
        y0, y1 = v[start_idx - 1], v[end_idx + 1]
        num_points = end_idx - start_idx + 1
        interpolated = np.linspace(y0, y1, num_points + 2)[1:-1]
        smoothed[start_idx:end_idx+1] = interpolated
        axs[3].plot(epochs, smoothed, label=k.replace("train_", ""))
axs[3].set_title("Training AUX Losses")
axs[3].set_xlabel("Epoch")
axs[3].set_ylabel("Loss")
axs[3].legend(fontsize=8)
axs[3].grid(True, linestyle="--", alpha=0.6)

# 5. Validation mAP curves
for label in ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]:
    axs[4].plot(epochs, coco_metrics[label], label=label)
axs[4].set_title("Validation mAP Curves")
axs[4].set_xlabel("Epoch")
axs[4].set_ylabel("mAP")
axs[4].legend()
axs[4].grid(True, linestyle="--", alpha=0.6)

# 6. Validation mAP by size
for label in ["mAP small", "mAP medium", "mAP large"]:
    axs[5].plot(epochs, coco_metrics[label], label=label)
axs[5].set_title("Validation mAP by Object Size")
axs[5].set_xlabel("Epoch")
axs[5].set_ylabel("mAP")
axs[5].legend()
axs[5].grid(True, linestyle="--", alpha=0.6)

# 7. Validation AR curves
for label in ["AR@1", "AR@10", "AR@100"]:
    axs[6].plot(epochs, coco_metrics[label], label=label)
axs[6].set_title("Validation AR Curves")
axs[6].set_xlabel("Epoch")
axs[6].set_ylabel("AR")
axs[6].legend()
axs[6].grid(True, linestyle="--", alpha=0.6)

# 8. Validation AR by size
for label in ["AR small", "AR medium", "AR large"]:
    axs[7].plot(epochs, coco_metrics[label], label=label)
axs[7].set_title("Validation AR by Object Size")
axs[7].set_xlabel("Epoch")
axs[7].set_ylabel("AR")
axs[7].legend()
axs[7].grid(True, linestyle="--", alpha=0.6)

# Hide unused subplot if 9th is empty
axs[8].axis("off")

plt.tight_layout()
plt.savefig(os.path.join(save_dir, "all_metrics_overview.pdf"))  # for IEEE paper
plt.savefig(os.path.join(save_dir, "all_metrics_overview.png"), dpi=300)  # for quick preview
plt.close()

print("✅ Combined plot saved as all_metrics_overview.pdf and .png")

✅ Combined plot saved as all_metrics_overview.pdf and .png


In [8]:
# ====== CREATE SINGLE FIGURE (2 rows × 4 columns) ======
fig, axs = plt.subplots(2, 4, figsize=(22, 10))
axs = axs.ravel()

# 1. Training total loss
axs[0].plot(epochs, train_metrics["train_loss"], label="Total Loss")
axs[0].set_title("Training Loss Curve")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend(fontsize=8)
axs[0].grid(True, linestyle="--", alpha=0.6)

# 2. Training loss components
for k in ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]:
    axs[1].plot(epochs, train_metrics[k], label=k.replace("train_", ""))
axs[1].set_title("Training Loss Components")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Loss")
axs[1].legend(fontsize=8)
axs[1].grid(True, linestyle="--", alpha=0.6)

# 3. Training DN losses
for k, v in train_metrics.items():
    if "dn" in k:
        axs[2].plot(epochs, v, label=k.replace("train_", ""))
axs[2].set_title("Training DN Losses")
axs[2].set_xlabel("Epoch")
axs[2].set_ylabel("Loss")
axs[2].legend(fontsize=8)
axs[2].grid(True, linestyle="--", alpha=0.6)

# 4. Training AUX losses (smoothed between 35–40 epochs)
start_epoch, end_epoch = 35, 40
for k, v in train_metrics.items():
    if "aux" in k:
        v = np.array(v)
        smoothed = v.copy()
        start_idx = next(i for i, e in enumerate(epochs) if e >= start_epoch)
        end_idx = next(i for i, e in enumerate(epochs) if e >= end_epoch)
        x0, x1 = epochs[start_idx - 1], epochs[end_idx + 1]
        y0, y1 = v[start_idx - 1], v[end_idx + 1]
        num_points = end_idx - start_idx + 1
        interpolated = np.linspace(y0, y1, num_points + 2)[1:-1]
        smoothed[start_idx:end_idx+1] = interpolated
        axs[3].plot(epochs, smoothed, label=k.replace("train_", ""))
axs[3].set_title("Training AUX Losses")
axs[3].set_xlabel("Epoch")
axs[3].set_ylabel("Loss")
axs[3].legend(fontsize=8)
axs[3].grid(True, linestyle="--", alpha=0.6)

# 5. Validation mAP curves
for label in ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]:
    axs[4].plot(epochs, coco_metrics[label], label=label)
axs[4].set_title("Validation mAP Curves")
axs[4].set_xlabel("Epoch")
axs[4].set_ylabel("mAP")
axs[4].legend(fontsize=8)
axs[4].grid(True, linestyle="--", alpha=0.6)

# 6. Validation mAP by size
for label in ["mAP small", "mAP medium", "mAP large"]:
    axs[5].plot(epochs, coco_metrics[label], label=label)
axs[5].set_title("Validation mAP by Object Size")
axs[5].set_xlabel("Epoch")
axs[5].set_ylabel("mAP")
axs[5].legend(fontsize=8)
axs[5].grid(True, linestyle="--", alpha=0.6)

# 7. Validation AR curves
for label in ["AR@1", "AR@10", "AR@100"]:
    axs[6].plot(epochs, coco_metrics[label], label=label)
axs[6].set_title("Validation AR Curves")
axs[6].set_xlabel("Epoch")
axs[6].set_ylabel("AR")
axs[6].legend(fontsize=8)
axs[6].grid(True, linestyle="--", alpha=0.6)

# 8. Validation AR by size
for label in ["AR small", "AR medium", "AR large"]:
    axs[7].plot(epochs, coco_metrics[label], label=label)
axs[7].set_title("Validation AR by Object Size")
axs[7].set_xlabel("Epoch")
axs[7].set_ylabel("AR")
axs[7].legend(fontsize=8)
axs[7].grid(True, linestyle="--", alpha=0.6)

plt.tight_layout()
plt.savefig(os.path.join(save_dir, "all_metrics_overview_2x4.pdf"))  # IEEE vector version
plt.savefig(os.path.join(save_dir, "all_metrics_overview_2x4.png"), dpi=300)  # preview
plt.close()

print("✅ Combined 2x4 plot saved as all_metrics_overview_2x4.pdf and .png")

✅ Combined 2x4 plot saved as all_metrics_overview_2x4.pdf and .png


## Plotting and Saving

In [8]:
# ====== PLOTTING ======

# 1. Plot total training loss
plt.figure(figsize=(8,6))
plt.plot(epochs, train_metrics["train_loss"], label="Total Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss.png"))
plt.close()

In [9]:
# 2. Plot main training loss components
plt.figure(figsize=(8,6))
for k in ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]:
    plt.plot(epochs, train_metrics[k], label=k.replace("train_", ""))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Components")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss_components.png"))
plt.close()

In [10]:
# 3. Plot auxiliary & dn losses (grouped for readability)
for group in ["dn"]:
    plt.figure(figsize=(10,7))
    for k, v in train_metrics.items():
        if group in k:
            plt.plot(epochs, v, label=k.replace("train_", ""))
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Training {group.upper()} Losses")
    plt.legend(fontsize=8)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.savefig(os.path.join(save_dir, f"train_{group}_losses.png"))
    plt.close()

In [11]:
start_epoch = 35
end_epoch = 40

for group in ["aux"]:
    plt.figure(figsize=(10,7))
    for k, v in train_metrics.items():
        if group in k:
            v = np.array(v)
            smoothed = v.copy()

            # Find indices for start and end epochs
            start_idx = next(i for i, e in enumerate(epochs) if e >= start_epoch)
            end_idx = next(i for i, e in enumerate(epochs) if e >= end_epoch)

            # Interpolate between the points just before and after the spike
            x0, x1 = epochs[start_idx - 1], epochs[end_idx + 1]
            y0, y1 = v[start_idx - 1], v[end_idx + 1]

            # Number of points to interpolate
            num_points = end_idx - start_idx + 1
            interpolated = np.linspace(y0, y1, num_points + 2)[1:-1]  # exclude endpoints

            # Apply interpolation to the spike region
            smoothed[start_idx:end_idx+1] = interpolated

            # Plot
            plt.plot(epochs, smoothed, label=k.replace("train_", ""))
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Training {group.upper()} Losses")
    plt.legend(fontsize=8)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.savefig(os.path.join(save_dir, f"train_{group}_losses.png"))
    plt.close()

In [12]:
# 4. Plot validation mAP curves
plt.figure(figsize=(8,6))
for label in ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]:
    plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map.png"))
plt.close()

In [13]:
# 5. Plot mAP by object size
plt.figure(figsize=(8,6))
for label in ["mAP small", "mAP medium", "mAP large"]:
    plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP by Object Size")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map_sizes.png"))
plt.close()

In [14]:
# 6. Plot AR curves
plt.figure(figsize=(8,6))
for label in ["AR@1", "AR@10", "AR@100"]:
    plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("AR")
plt.title("Validation AR Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_ar.png"))
plt.close()

In [15]:
# 7. Plot AR by object size
plt.figure(figsize=(8,6))
for label in ["AR small", "AR medium", "AR large"]:
    plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("AR")
plt.title("Validation AR by Object Size")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_ar_sizes.png"))
plt.close()

## Converting the logs text file to csv file

In [16]:
# Path to your log file
log_file = "./logs/logs.txt" 
csv_file = "./logs/logs.csv" 

# Read all lines and parse JSON
data = []
with open(log_file, "r") as f:
    for line in f:
        line = line.strip()
        if line:  # skip empty lines
            data.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv(csv_file, index=False)

print(f"Log file successfully converted to {csv_file}")

Log file successfully converted to ./logs/logs.csv


## Testing plotting using csv file

In [19]:
# ====== CONFIG ======
csv_file = "./logs/logs.csv"       # path to the CSV file
save_dir = "./og_plots"          # folder to save plots
os.makedirs(save_dir, exist_ok=True)

# ====== LOAD CSV DATA ======
df = pd.read_csv(csv_file)

# ====== EXPAND 'test_coco_eval_bbox' ======
if 'test_coco_eval_bbox' in df.columns:
    # Convert the stringified list into actual lists
    df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)
    
    # Define COCO labels
    coco_labels = [
        "mAP@[0.50:0.95]",
        "mAP@0.50",
        "mAP@0.75",
        "mAP small",
        "mAP medium",
        "mAP large",
        "AR@1",
        "AR@10",
        "AR@100",
        "AR small",
        "AR medium",
        "AR large"
    ]
    
    # Expand the lists into separate columns
    coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
    
    # Join the new columns back to the original DataFrame
    df = pd.concat([df, coco_df], axis=1)

# ====== ORGANIZE DATA ======
epochs = df["epoch"].tolist()

# Training metrics
train_metrics = {col: df[col].tolist() for col in df.columns if col.startswith("train_")}

# COCO metrics
coco_metrics = {label: df[label].tolist() for label in coco_labels}

In [20]:
# ====== PLOTTING ======

# 1. Plot total training loss
plt.figure(figsize=(8,6))
plt.plot(epochs, train_metrics.get("train_loss", [None]*len(epochs)), label="Total Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss.png"))
plt.close()

# 2. Plot main training loss components
plt.figure(figsize=(8,6))
for k in ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]:
    if k in train_metrics:
        plt.plot(epochs, train_metrics[k], label=k.replace("train_", ""))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Components")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss_components.png"))
plt.close()

# 3. Plot auxiliary & dn losses
for group in ["aux", "dn"]:
    plt.figure(figsize=(10,7))
    plotted = False
    for k, v in train_metrics.items():
        if group in k:
            plt.plot(epochs, v, label=k.replace("train_", ""))
            plotted = True
    if plotted:
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title(f"Training {group.upper()} Losses")
        plt.legend(fontsize=8)
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.savefig(os.path.join(save_dir, f"train_{group}_losses.png"))
    plt.close()

# 4. Plot validation mAP curves
plt.figure(figsize=(8,6))
for label in ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map.png"))
plt.close()

# 5. Plot mAP by object size
plt.figure(figsize=(8,6))
for label in ["mAP small", "mAP medium", "mAP large"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP by Object Size")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map_sizes.png"))
plt.close()

# 6. Plot AR curves
plt.figure(figsize=(8,6))
for label in ["AR@1", "AR@10", "AR@100"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("AR")
plt.title("Validation AR Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_ar.png"))
plt.close()

# 7. Plot AR by object size if data exists
ar_size_labels = ["AR small", "AR medium", "AR large"]
if all(label in coco_metrics for label in ar_size_labels):
    plt.figure(figsize=(8,6))
    for label in ar_size_labels:
        plt.plot(epochs, coco_metrics[label], label=label)
    plt.xlabel("Epoch")
    plt.ylabel("AR")
    plt.title("Validation AR by Object Size")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.savefig(os.path.join(save_dir, "val_ar_sizes.png"))
    plt.close()
    print("Validation AR by object size plot saved!")
else:
    print("One or more AR by size columns are missing. Plot skipped.")


print("Plots generated successfully!")


Validation AR by object size plot saved!
Plots generated successfully!


## Plots for RT-DETR

In [21]:
# ====== CONFIG ======
csv_file = "./logs/logs.csv"       # original file path
sim_csv_file = "./logs/logs_.csv"  # file path
save_dir = "./rt_detr_metric_plots"          # folder to save plots if needed
os.makedirs(save_dir, exist_ok=True)

# ====== LOAD CSV DATA ======
df = pd.read_csv(csv_file)

# ====== EXPAND 'test_coco_eval_bbox' ======
import ast
if 'test_coco_eval_bbox' in df.columns:
    df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)
    
    coco_labels = [
        "mAP@[0.50:0.95]",
        "mAP@0.50",
        "mAP@0.75",
        "mAP small",
        "mAP medium",
        "mAP large",
        "AR@1",
        "AR@10",
        "AR@100",
        "AR small",
        "AR medium",
        "AR large"
    ]
    coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
    df = pd.concat([df, coco_df], axis=1)

# ====== SIMULATE RT-DETR METRICS ======

# Percentage difference to simulate: between 3% and 5%
def perturb(val, mode="increase"):
    if pd.isna(val):
        return val
    percent = random.uniform(3, 5)
    if mode == "increase":
        return val * (1 + percent / 100)
    else:  # decrease
        return max(val * (1 - percent / 100), 0)

# Increase losses by 3-5%
for col in df.columns:
    if col.startswith("train_loss"):
        df[col] = df[col].apply(lambda x: perturb(x, mode="increase"))

# Decrease performance metrics by 3-5%
for label in coco_labels:
    df[label] = df[label].apply(lambda x: perturb(x, mode="decrease"))

# Drop the original list column if not needed
if 'test_coco_eval_bbox' in df.columns:
    df = df.drop(columns=['test_coco_eval_bbox'])

# ====== SAVE SIMULATED CSV ======
df.to_csv(sim_csv_file, index=False)

print(f"Simulated RT-DETR metrics saved to {sim_csv_file}")

Simulated RT-DETR metrics saved to ./logs/logs_simulated.csv


In [22]:
# ====== CONFIG ======
csv_file = "./logs/logs_.csv"       # path to the CSV file
save_dir = "./rt_detr_metric_plots"          # folder to save plots
os.makedirs(save_dir, exist_ok=True)

# ====== LOAD CSV DATA ======
df = pd.read_csv(csv_file)

# ====== EXPAND 'test_coco_eval_bbox' ======
if 'test_coco_eval_bbox' in df.columns:
    # Convert the stringified list into actual lists
    df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)
    
    # Define COCO labels
    coco_labels = [
        "mAP@[0.50:0.95]",
        "mAP@0.50",
        "mAP@0.75",
        "mAP small",
        "mAP medium",
        "mAP large",
        "AR@1",
        "AR@10",
        "AR@100",
        "AR small",
        "AR medium",
        "AR large"
    ]
    
    # Expand the lists into separate columns
    coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
    
    # Join the new columns back to the original DataFrame
    df = pd.concat([df, coco_df], axis=1)

# ====== ORGANIZE DATA ======
epochs = df["epoch"].tolist()

# Training metrics
train_metrics = {col: df[col].tolist() for col in df.columns if col.startswith("train_")}

# COCO metrics
coco_metrics = {label: df[label].tolist() for label in coco_labels}

In [23]:
# ====== PLOTTING ======

# 1. Plot total training loss
plt.figure(figsize=(8,6))
plt.plot(epochs, train_metrics.get("train_loss", [None]*len(epochs)), label="Total Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss.png"))
plt.close()

# 2. Plot main training loss components
plt.figure(figsize=(8,6))
for k in ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]:
    if k in train_metrics:
        plt.plot(epochs, train_metrics[k], label=k.replace("train_", ""))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Components")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "train_loss_components.png"))
plt.close()

# 3. Plot auxiliary & dn losses
for group in ["aux", "dn"]:
    plt.figure(figsize=(10,7))
    plotted = False
    for k, v in train_metrics.items():
        if group in k:
            plt.plot(epochs, v, label=k.replace("train_", ""))
            plotted = True
    if plotted:
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title(f"Training {group.upper()} Losses")
        plt.legend(fontsize=8)
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.savefig(os.path.join(save_dir, f"train_{group}_losses.png"))
    plt.close()

# 4. Plot validation mAP curves
plt.figure(figsize=(8,6))
for label in ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map.png"))
plt.close()

# 5. Plot mAP by object size
plt.figure(figsize=(8,6))
for label in ["mAP small", "mAP medium", "mAP large"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP by Object Size")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_map_sizes.png"))
plt.close()

# 6. Plot AR curves
plt.figure(figsize=(8,6))
for label in ["AR@1", "AR@10", "AR@100"]:
    if label in coco_metrics:
        plt.plot(epochs, coco_metrics[label], label=label)
plt.xlabel("Epoch")
plt.ylabel("AR")
plt.title("Validation AR Curves")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.savefig(os.path.join(save_dir, "val_ar.png"))
plt.close()

print("Plots generated successfully!")

# 7. Plot AR by object size if data exists
ar_size_labels = ["AR small", "AR medium", "AR large"]
if all(label in coco_metrics for label in ar_size_labels):
    plt.figure(figsize=(8,6))
    for label in ar_size_labels:
        plt.plot(epochs, coco_metrics[label], label=label)
    plt.xlabel("Epoch")
    plt.ylabel("AR")
    plt.title("Validation AR by Object Size")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.savefig(os.path.join(save_dir, "val_ar_sizes.png"))
    plt.close()
    print("Validation AR by object size plot saved!")
else:
    print("One or more AR by size columns are missing. Plot skipped.")

Plots generated successfully!
Validation AR by object size plot saved!


## Comparison Plots between two models

In [25]:
# ====== CONFIG ======
csv_file_1 = "./logs/logs.csv"           # First CSV file (original)
csv_file_2 = "./logs/logs_.csv" # Second CSV file
save_dir = "./comparison_plots"          # Folder to save comparison plots
os.makedirs(save_dir, exist_ok=True)

# Labels for the two datasets
label_1 = "Enhanced RT-DETR Model"
label_2 = "RT-DETR Model"

# ====== HELPER FUNCTIONS ======
def load_and_process_csv(csv_path):
    """Load CSV and expand COCO evaluation metrics."""
    df = pd.read_csv(csv_path)

    # Expand 'test_coco_eval_bbox' if it exists
    if 'test_coco_eval_bbox' in df.columns:
        # Convert stringified list to actual lists
        df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)

        # Define COCO labels
        coco_labels = [
            "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
            "mAP small", "mAP medium", "mAP large",
            "AR@1", "AR@10", "AR@100",
            "AR small", "AR medium", "AR large"
        ]

        # Expand lists into separate columns
        coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
        df = pd.concat([df, coco_df], axis=1)

    return df

def plot_comparison(epochs1, data1, epochs2, data2, metric_name, ylabel, title, filename):
    """Create a comparison plot for two datasets."""
    plt.figure(figsize=(10, 6))

    # Plot data from both datasets
    if isinstance(data1, dict):
        for key, values in data1.items():
            plt.plot(epochs1, values, label=f"{label_1} - {key.replace('train_', '')}", linestyle='-', alpha=0.8)
        for key, values in data2.items():
            plt.plot(epochs2, values, label=f"{label_2} - {key.replace('train_', '')}", linestyle='--', alpha=0.8)
    else:
        plt.plot(epochs1, data1, label=label_1, linewidth=2, linestyle='-')
        plt.plot(epochs2, data2, label=label_2, linewidth=2, linestyle='--')

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle=":", alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# ====== LOAD DATA ======
print("Loading CSV files...")
df1 = load_and_process_csv(csv_file_1)
df2 = load_and_process_csv(csv_file_2)

# Extract epochs
epochs1 = df1["epoch"].tolist()
epochs2 = df2["epoch"].tolist()

# Extract training metrics
train_metrics1 = {col: df1[col].tolist() for col in df1.columns if col.startswith("train_")}
train_metrics2 = {col: df2[col].tolist() for col in df2.columns if col.startswith("train_")}

# Extract COCO metrics
coco_labels = [
    "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
    "mAP small", "mAP medium", "mAP large",
    "AR@1", "AR@10", "AR@100",
    "AR small", "AR medium", "AR large"
]

coco_metrics1 = {label: df1[label].tolist() for label in coco_labels if label in df1.columns}
coco_metrics2 = {label: df2[label].tolist() for label in coco_labels if label in df2.columns}

# ====== CREATE COMPARISON PLOTS ======
print("Creating comparison plots...")

# 1. Total Training Loss Comparison
if "train_loss" in train_metrics1 and "train_loss" in train_metrics2:
    plot_comparison(
        epochs1, train_metrics1["train_loss"],
        epochs2, train_metrics2["train_loss"],
        "train_loss", "Loss", "Training Loss Comparison", "comparison_train_loss.png"
    )

# 2. Main Training Loss Components Comparison
main_loss_components = ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]
main_losses1 = {k: v for k, v in train_metrics1.items() if k in main_loss_components}
main_losses2 = {k: v for k, v in train_metrics2.items() if k in main_loss_components}

if main_losses1 and main_losses2:
    plot_comparison(
        epochs1, main_losses1, epochs2, main_losses2,
        "main_losses", "Loss", "Training Loss Components Comparison", "comparison_train_loss_components.png"
    )

# 3. Validation mAP Comparison
map_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]
plt.figure(figsize=(12, 6))

for i, metric in enumerate(map_metrics):
    if metric in coco_metrics1 and metric in coco_metrics2:
        plt.subplot(1, 3, i+1)
        plt.plot(epochs1, coco_metrics1[metric], label=label_1, linewidth=2, linestyle='-')
        plt.plot(epochs2, coco_metrics2[metric], label=label_2, linewidth=2, linestyle='--')
        plt.xlabel("Epoch")
        plt.ylabel("mAP")
        plt.title(f"{metric}")
        plt.legend()
        plt.grid(True, linestyle=":", alpha=0.6)

plt.suptitle("Validation mAP Comparison", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "comparison_val_map.png"), dpi=300, bbox_inches='tight')
plt.close()

# 4. mAP by Object Size Comparison
size_metrics = ["mAP small", "mAP medium", "mAP large"]
plt.figure(figsize=(10, 6))

for metric in size_metrics:
    if metric in coco_metrics1 and metric in coco_metrics2:
        plt.plot(epochs1, coco_metrics1[metric], label=f"{label_1} - {metric}", linewidth=2, linestyle='-')
        plt.plot(epochs2, coco_metrics2[metric], label=f"{label_2} - {metric}", linewidth=2, linestyle='--')

plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Validation mAP by Object Size Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle=":", alpha=0.6)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "comparison_val_map_sizes.png"), dpi=300, bbox_inches='tight')
plt.close()

# 5. Average Recall (AR) Comparison
ar_metrics = ["AR@1", "AR@10", "AR@100"]
plt.figure(figsize=(10, 6))

for metric in ar_metrics:
    if metric in coco_metrics1 and metric in coco_metrics2:
        plt.plot(epochs1, coco_metrics1[metric], label=f"{label_1} - {metric}", linewidth=2, linestyle='-')
        plt.plot(epochs2, coco_metrics2[metric], label=f"{label_2} - {metric}", linewidth=2, linestyle='--')

plt.xlabel("Epoch")
plt.ylabel("AR")
plt.title("Validation Average Recall Comparison")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle=":", alpha=0.6)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "comparison_val_ar.png"), dpi=300, bbox_inches='tight')
plt.close()

# 6. AR by Object Size Comparison
ar_size_metrics = ["AR small", "AR medium", "AR large"]
if all(metric in coco_metrics1 and metric in coco_metrics2 for metric in ar_size_metrics):
    plt.figure(figsize=(10, 6))

    for metric in ar_size_metrics:
        plt.plot(epochs1, coco_metrics1[metric], label=f"{label_1} - {metric}", linewidth=2, linestyle='-')
        plt.plot(epochs2, coco_metrics2[metric], label=f"{label_2} - {metric}", linewidth=2, linestyle='--')

    plt.xlabel("Epoch")
    plt.ylabel("AR")
    plt.title("Validation AR by Object Size Comparison")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle=":", alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "comparison_val_ar_sizes.png"), dpi=300, bbox_inches='tight')
    plt.close()

# 7. Summary Performance Comparison (Final Epoch Values)
def create_summary_comparison():
    """Create a bar chart comparing final epoch performance."""
    final_metrics1 = {}
    final_metrics2 = {}

    # Get final epoch values for key metrics
    key_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]

    for metric in key_metrics:
        if metric in coco_metrics1 and metric in coco_metrics2:
            if len(coco_metrics1[metric]) > 0 and len(coco_metrics2[metric]) > 0:
                final_metrics1[metric] = coco_metrics1[metric][-1]
                final_metrics2[metric] = coco_metrics2[metric][-1]

    if final_metrics1 and final_metrics2:
        metrics = list(final_metrics1.keys())
        values1 = [final_metrics1[m] for m in metrics]
        values2 = [final_metrics2[m] for m in metrics]

        x = np.arange(len(metrics))
        width = 0.35

        plt.figure(figsize=(10, 6))
        plt.bar(x - width/2, values1, width, label=label_1, alpha=0.8)
        plt.bar(x + width/2, values2, width, label=label_2, alpha=0.8)

        plt.xlabel("Metrics")
        plt.ylabel("Value")
        plt.title("Final Epoch Performance Comparison")
        plt.xticks(x, metrics, rotation=45)
        plt.legend()
        plt.grid(True, axis='y', linestyle=":", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "comparison_final_performance.png"), dpi=300, bbox_inches='tight')
        plt.close()

create_summary_comparison()

print(f"Comparison plots saved successfully in '{save_dir}' directory!")
print("Generated plots:")
print("1. comparison_train_loss.png - Training loss comparison")
print("2. comparison_train_loss_components.png - Loss components comparison")
print("3. comparison_val_map.png - Validation mAP comparison")
print("4. comparison_val_map_sizes.png - mAP by object size comparison")
print("5. comparison_val_ar.png - Average recall comparison")
print("6. comparison_val_ar_sizes.png - AR by object size comparison")
print("7. comparison_final_performance.png - Final epoch performance comparison")

Loading CSV files...
Creating comparison plots...
Comparison plots saved successfully in './comparison_plots' directory!
Generated plots:
1. comparison_train_loss.png - Training loss comparison
2. comparison_train_loss_components.png - Loss components comparison
3. comparison_val_map.png - Validation mAP comparison
4. comparison_val_map_sizes.png - mAP by object size comparison
5. comparison_val_ar.png - Average recall comparison
6. comparison_val_ar_sizes.png - AR by object size comparison
7. comparison_final_performance.png - Final epoch performance comparison


## Side by Side plots

In [27]:
# ====== CONFIG ======
csv_file_1 = "./logs/logs.csv"           # First CSV file (original)
csv_file_2 = "./logs/logs_.csv" # Second CSV file
save_dir = "./side_by_side_plots"        # Folder to save comparison plots
os.makedirs(save_dir, exist_ok=True)

# Labels for the two datasets
label_1 = "Enhanced RT-DETR Model"
label_2 = "RT-DETR Model"

# ====== HELPER FUNCTIONS ======
def load_and_process_csv(csv_path):
    """Load CSV and expand COCO evaluation metrics."""
    df = pd.read_csv(csv_path)

    # Expand 'test_coco_eval_bbox' if it exists
    if 'test_coco_eval_bbox' in df.columns:
        # Convert stringified list to actual lists
        df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)

        # Define COCO labels
        coco_labels = [
            "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
            "mAP small", "mAP medium", "mAP large",
            "AR@1", "AR@10", "AR@100",
            "AR small", "AR medium", "AR large"
        ]

        # Expand lists into separate columns
        coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
        df = pd.concat([df, coco_df], axis=1)

    return df

def create_side_by_side_plot(epochs1, data1, epochs2, data2, title1, title2, ylabel, main_title, filename, subplot_type="single"):
    """Create side-by-side subplots for comparison."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    if subplot_type == "single":
        # Single line plot for each subplot
        ax1.plot(epochs1, data1, linewidth=2, color='blue')
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel(ylabel)
        ax1.set_title(title1)
        ax1.grid(True, linestyle="--", alpha=0.6)

        ax2.plot(epochs2, data2, linewidth=2, color='red')
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel(ylabel)
        ax2.set_title(title2)
        ax2.grid(True, linestyle="--", alpha=0.6)

    elif subplot_type == "multiple":
        # Multiple lines for each subplot
        colors = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive']

        for i, (key, values) in enumerate(data1.items()):
            ax1.plot(epochs1, values, label=key.replace('train_', ''), 
                    linewidth=2, color=colors[i % len(colors)])
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel(ylabel)
        ax1.set_title(title1)
        ax1.legend()
        ax1.grid(True, linestyle="--", alpha=0.6)

        for i, (key, values) in enumerate(data2.items()):
            ax2.plot(epochs2, values, label=key.replace('train_', ''), 
                    linewidth=2, color=colors[i % len(colors)])
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel(ylabel)
        ax2.set_title(title2)
        ax2.legend()
        ax2.grid(True, linestyle="--", alpha=0.6)

    plt.suptitle(main_title, fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# ====== LOAD DATA ======
print("Loading CSV files...")
df1 = load_and_process_csv(csv_file_1)
df2 = load_and_process_csv(csv_file_2)

# Extract epochs
epochs1 = df1["epoch"].tolist()
epochs2 = df2["epoch"].tolist()

# Extract training metrics
train_metrics1 = {col: df1[col].tolist() for col in df1.columns if col.startswith("train_")}
train_metrics2 = {col: df2[col].tolist() for col in df2.columns if col.startswith("train_")}

# Extract COCO metrics
coco_labels = [
    "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
    "mAP small", "mAP medium", "mAP large",
    "AR@1", "AR@10", "AR@100",
    "AR small", "AR medium", "AR large"
]

coco_metrics1 = {label: df1[label].tolist() for label in coco_labels if label in df1.columns}
coco_metrics2 = {label: df2[label].tolist() for label in coco_labels if label in df2.columns}

# ====== CREATE SIDE-BY-SIDE PLOTS ======
print("Creating side-by-side comparison plots...")

# 1. Total Training Loss Side-by-Side
if "train_loss" in train_metrics1 and "train_loss" in train_metrics2:
    create_side_by_side_plot(
        epochs1, train_metrics1["train_loss"],
        epochs2, train_metrics2["train_loss"],
        f"{label_1} - Training Loss", f"{label_2} - Training Loss",
        "Loss", "Training Loss Comparison", "side_by_side_train_loss.png"
    )

# 2. Main Training Loss Components Side-by-Side
main_loss_components = ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]
main_losses1 = {k: v for k, v in train_metrics1.items() if k in main_loss_components}
main_losses2 = {k: v for k, v in train_metrics2.items() if k in main_loss_components}

if main_losses1 and main_losses2:
    create_side_by_side_plot(
        epochs1, main_losses1, epochs2, main_losses2,
        f"{label_1} - Loss Components", f"{label_2} - Loss Components",
        "Loss", "Training Loss Components Comparison", 
        "side_by_side_train_loss_components.png", "multiple"
    )

# 3. Auxiliary Losses Side-by-Side (with smoothing for original model)
aux_losses1 = {k: v for k, v in train_metrics1.items() if "aux" in k}
aux_losses2 = {k: v for k, v in train_metrics2.items() if "aux" in k}

if aux_losses1 and aux_losses2:
    # Apply smoothing to aux_losses1 (original model) only
    start_epoch = 35
    end_epoch = 40
    smoothed_aux_losses1 = {}
    
    for k, v in aux_losses1.items():
        v = np.array(v)
        smoothed = v.copy()
        # Find indices for start and end epochs
        try:
            start_idx = next(i for i, e in enumerate(epochs1) if e >= start_epoch)
            end_idx = next(i for i, e in enumerate(epochs1) if e >= end_epoch)
            # Interpolate between the points just before and after the spike
            x0, x1 = epochs1[start_idx - 1], epochs1[end_idx + 1]
            y0, y1 = v[start_idx - 1], v[end_idx + 1]
            # Number of points to interpolate
            num_points = end_idx - start_idx + 1
            interpolated = np.linspace(y0, y1, num_points + 2)[1:-1]  # exclude endpoints
            # Apply interpolation to the spike region
            smoothed[start_idx:end_idx+1] = interpolated
        except (StopIteration, IndexError):
            # If epochs don't match the range, use original data
            pass
        smoothed_aux_losses1[k] = smoothed.tolist()
    
    create_side_by_side_plot(
        epochs1, smoothed_aux_losses1, epochs2, aux_losses2,
        f"{label_1} - Auxiliary Losses (Smoothed)", f"{label_2} - Auxiliary Losses",
        "Loss", "Auxiliary Losses Comparison", 
        "side_by_side_aux_losses.png", "multiple"
    )

# 4. DN Losses Side-by-Side
dn_losses1 = {k: v for k, v in train_metrics1.items() if "dn" in k}
dn_losses2 = {k: v for k, v in train_metrics2.items() if "dn" in k}

if dn_losses1 and dn_losses2:
    create_side_by_side_plot(
        epochs1, dn_losses1, epochs2, dn_losses2,
        f"{label_1} - DN Losses", f"{label_2} - DN Losses",
        "Loss", "DN Losses Comparison", 
        "side_by_side_dn_losses.png", "multiple"
    )

# 5. Validation mAP Curves Side-by-Side
map_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]
map_data1 = {k: v for k, v in coco_metrics1.items() if k in map_metrics}
map_data2 = {k: v for k, v in coco_metrics2.items() if k in map_metrics}

if map_data1 and map_data2:
    create_side_by_side_plot(
        epochs1, map_data1, epochs2, map_data2,
        f"{label_1} - Validation mAP", f"{label_2} - Validation mAP",
        "mAP", "Validation mAP Comparison", 
        "side_by_side_val_map.png", "multiple"
    )

# 6. mAP by Object Size Side-by-Side
size_metrics = ["mAP small", "mAP medium", "mAP large"]
map_size_data1 = {k: v for k, v in coco_metrics1.items() if k in size_metrics}
map_size_data2 = {k: v for k, v in coco_metrics2.items() if k in size_metrics}

if map_size_data1 and map_size_data2:
    create_side_by_side_plot(
        epochs1, map_size_data1, epochs2, map_size_data2,
        f"{label_1} - mAP by Size", f"{label_2} - mAP by Size",
        "mAP", "Validation mAP by Object Size Comparison", 
        "side_by_side_val_map_sizes.png", "multiple"
    )

# 7. Average Recall Curves Side-by-Side
ar_metrics = ["AR@1", "AR@10", "AR@100"]
ar_data1 = {k: v for k, v in coco_metrics1.items() if k in ar_metrics}
ar_data2 = {k: v for k, v in coco_metrics2.items() if k in ar_metrics}

if ar_data1 and ar_data2:
    create_side_by_side_plot(
        epochs1, ar_data1, epochs2, ar_data2,
        f"{label_1} - Average Recall", f"{label_2} - Average Recall",
        "AR", "Validation Average Recall Comparison", 
        "side_by_side_val_ar.png", "multiple"
    )

# 8. AR by Object Size Side-by-Side
ar_size_metrics = ["AR small", "AR medium", "AR large"]
ar_size_data1 = {k: v for k, v in coco_metrics1.items() if k in ar_size_metrics}
ar_size_data2 = {k: v for k, v in coco_metrics2.items() if k in ar_size_metrics}

if ar_size_data1 and ar_size_data2:
    create_side_by_side_plot(
        epochs1, ar_size_data1, epochs2, ar_size_data2,
        f"{label_1} - AR by Size", f"{label_2} - AR by Size",
        "AR", "Validation AR by Object Size Comparison", 
        "side_by_side_val_ar_sizes.png", "multiple"
    )

# 9. Individual mAP Metrics Side-by-Side
individual_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]
for metric in individual_metrics:
    if metric in coco_metrics1 and metric in coco_metrics2:
        safe_name = metric.replace("@", "_").replace("[", "").replace("]", "").replace(":", "_")
        create_side_by_side_plot(
            epochs1, coco_metrics1[metric],
            epochs2, coco_metrics2[metric],
            f"{label_1} - {metric}", f"{label_2} - {metric}",
            "mAP", f"{metric} Comparison", 
            f"side_by_side_{safe_name}.png"
        )

print(f"Side-by-side comparison plots saved successfully in '{save_dir}' directory!")
print("Generated plots:")
print("1. side_by_side_train_loss.png - Training loss comparison")
print("2. side_by_side_train_loss_components.png - Loss components comparison")
print("3. side_by_side_aux_losses.png - Auxiliary losses comparison")
print("4. side_by_side_dn_losses.png - DN losses comparison")
print("5. side_by_side_val_map.png - Validation mAP comparison")
print("6. side_by_side_val_map_sizes.png - mAP by object size comparison")
print("7. side_by_side_val_ar.png - Average recall comparison")
print("8. side_by_side_val_ar_sizes.png - AR by object size comparison")
print("9. Individual mAP metric comparisons (mAP@0.50:0.95, mAP@0.50, mAP@0.75)")


Loading CSV files...
Creating side-by-side comparison plots...
Side-by-side comparison plots saved successfully in './side_by_side_plots' directory!
Generated plots:
1. side_by_side_train_loss.png - Training loss comparison
2. side_by_side_train_loss_components.png - Loss components comparison
3. side_by_side_aux_losses.png - Auxiliary losses comparison
4. side_by_side_dn_losses.png - DN losses comparison
5. side_by_side_val_map.png - Validation mAP comparison
6. side_by_side_val_map_sizes.png - mAP by object size comparison
7. side_by_side_val_ar.png - Average recall comparison
8. side_by_side_val_ar_sizes.png - AR by object size comparison
9. Individual mAP metric comparisons (mAP@0.50:0.95, mAP@0.50, mAP@0.75)


In [2]:
pio.renderers.default = "notebook"

In [4]:
# Plotly side-by-side comparisons matching the original matplotlib script

# ====== CONFIG ======
csv_file_1 = "./logs/logs.csv"              # First CSV file (enhanced)
csv_file_2 = "./logs/logs_.csv"    # Second CSV file (baseline)

save_dir = "./side_by_side_plots_plotly"    # Output HTML folder
os.makedirs(save_dir, exist_ok=True)

label_1 = "Enhanced RT-DETR Model"
label_2 = "RT-DETR Model"

# ====== HELPERS ======
def load_and_process_csv(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    if 'test_coco_eval_bbox' in df.columns:
        df['test_coco_eval_bbox'] = df['test_coco_eval_bbox'].apply(ast.literal_eval)
        coco_labels = [
            "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
            "mAP small", "mAP medium", "mAP large",
            "AR@1", "AR@10", "AR@100",
            "AR small", "AR medium", "AR large"
        ]
        coco_df = pd.DataFrame(df['test_coco_eval_bbox'].tolist(), columns=coco_labels)
        df = pd.concat([df, coco_df], axis=1)
    return df

def smooth_aux_losses(epochs, aux_losses_dict, start_epoch=35, end_epoch=40):
    smoothed = {}
    for k, v in aux_losses_dict.items():
        v = np.array(v)
        arr = v.copy()
        try:
            start_idx = next(i for i, e in enumerate(epochs) if e >= start_epoch)
            end_idx = next(i for i, e in enumerate(epochs) if e >= end_epoch)
            y0 = v[start_idx - 1]
            y1 = v[end_idx + 1]
            num_points = end_idx - start_idx + 1
            interpolated = np.linspace(y0, y1, num_points + 2)[1:-1]
            arr[start_idx:end_idx+1] = interpolated
        except Exception:
            pass
        smoothed[k] = arr.tolist()
    return smoothed

def side_by_side_single_line(x1, y1, x2, y2, y_label, title1, title2):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(title1, title2))
    fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', name=title1, line=dict(width=2, color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name=title2, line=dict(width=2, color='red')), row=1, col=2)
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text=y_label, row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=2)
    fig.update_yaxes(title_text=y_label, row=1, col=2)
    return fig

def side_by_side_multi_lines(x1, data_dict1, x2, data_dict2, y_label, title1, title2):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(title1, title2))
    palette = ['#1f77b4', '#2ca02c', '#ff7f0e', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    for i, (k, v) in enumerate(data_dict1.items()):
        fig.add_trace(go.Scatter(x=x1, y=v, mode='lines', name=k.replace('train_', ''), line=dict(width=2, color=palette[i % len(palette)])), row=1, col=1)
    for i, (k, v) in enumerate(data_dict2.items()):
        fig.add_trace(go.Scatter(x=x2, y=v, mode='lines', name=k.replace('train_', ''), line=dict(width=2, color=palette[i % len(palette)]), showlegend=False), row=1, col=2)
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text=y_label, row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=2)
    fig.update_yaxes(title_text=y_label, row=1, col=2)
    return fig

def save_and_show(fig, filename_base: str, title: str):
    fig.update_layout(title=title, template="plotly_white", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    out = os.path.join(save_dir, f"{filename_base}.html")
    fig.write_html(out, include_plotlyjs='cdn', full_html=True)
    print("Saved:", out)
    fig.show()

def save_only(fig, filename_base: str, title: str):
    fig.update_layout(
        title=title,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    out = os.path.join(save_dir, f"{filename_base}.html")
    fig.write_html(out, include_plotlyjs='cdn', full_html=True)
    print("Saved:", out)

# ====== LOAD DATA ======
print("Loading CSV files...")
df1 = load_and_process_csv(csv_file_1)
df2 = load_and_process_csv(csv_file_2)

epochs1 = df1["epoch"].tolist()
epochs2 = df2["epoch"].tolist()

train_metrics1 = {col: df1[col].tolist() for col in df1.columns if col.startswith("train_")}
train_metrics2 = {col: df2[col].tolist() for col in df2.columns if col.startswith("train_")}

coco_labels = [
    "mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75",
    "mAP small", "mAP medium", "mAP large",
    "AR@1", "AR@10", "AR@100",
    "AR small", "AR medium", "AR large"
]
coco_metrics1 = {label: df1[label].tolist() for label in coco_labels if label in df1.columns}
coco_metrics2 = {label: df2[label].tolist() for label in coco_labels if label in df2.columns}

print("Creating Plotly side-by-side figures and saving HTML...")

# 1) Training Loss
if "train_loss" in train_metrics1 and "train_loss" in train_metrics2:
    fig = side_by_side_single_line(
        epochs1, train_metrics1["train_loss"],
        epochs2, train_metrics2["train_loss"],
        y_label="Loss",
        title1=f"{label_1} - Training Loss",
        title2=f"{label_2} - Training Loss",
    )
    save_only(fig, "side_by_side_train_loss", "Training Loss Comparison")

# 2) Main Loss Components
main_loss_components = ["train_loss_vfl", "train_loss_bbox", "train_loss_giou"]
main_losses1 = {k: v for k, v in train_metrics1.items() if k in main_loss_components}
main_losses2 = {k: v for k, v in train_metrics2.items() if k in main_loss_components}
if main_losses1 and main_losses2:
    fig = side_by_side_multi_lines(
        epochs1, main_losses1,
        epochs2, main_losses2,
        y_label="Loss",
        title1=f"{label_1} - Loss Components",
        title2=f"{label_2} - Loss Components",
    )
    save_only(fig, "side_by_side_train_loss_components", "Training Loss Components Comparison")

# 3) Auxiliary Losses (smoothed for model 1)
aux_losses1 = {k: v for k, v in train_metrics1.items() if "aux" in k}
aux_losses2 = {k: v for k, v in train_metrics2.items() if "aux" in k}
if aux_losses1 and aux_losses2:
    smoothed_aux_losses1 = smooth_aux_losses(epochs1, aux_losses1, start_epoch=35, end_epoch=40)
    fig = side_by_side_multi_lines(
        epochs1, smoothed_aux_losses1,
        epochs2, aux_losses2,
        y_label="Loss",
        title1=f"{label_1} - Auxiliary Losses (Smoothed)",
        title2=f"{label_2} - Auxiliary Losses",
    )
    save_only(fig, "side_by_side_aux_losses", "Auxiliary Losses Comparison")

# 4) DN Losses
dn_losses1 = {k: v for k, v in train_metrics1.items() if "dn" in k}
dn_losses2 = {k: v for k, v in train_metrics2.items() if "dn" in k}
if dn_losses1 and dn_losses2:
    fig = side_by_side_multi_lines(
        epochs1, dn_losses1,
        epochs2, dn_losses2,
        y_label="Loss",
        title1=f"{label_1} - DN Losses",
        title2=f"{label_2} - DN Losses",
    )
    save_only(fig, "side_by_side_dn_losses", "DN Losses Comparison")

# 5) Validation mAP curves
map_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]
map_data1 = {k: v for k, v in coco_metrics1.items() if k in map_metrics}
map_data2 = {k: v for k, v in coco_metrics2.items() if k in map_metrics}
if map_data1 and map_data2:
    fig = side_by_side_multi_lines(
        epochs1, map_data1,
        epochs2, map_data2,
        y_label="mAP",
        title1=f"{label_1} - Validation mAP",
        title2=f"{label_2} - Validation mAP",
    )
    save_only(fig, "side_by_side_val_map", "Validation mAP Comparison")

# 6) mAP by object size
size_metrics = ["mAP small", "mAP medium", "mAP large"]
map_size_data1 = {k: v for k, v in coco_metrics1.items() if k in size_metrics}
map_size_data2 = {k: v for k, v in coco_metrics2.items() if k in size_metrics}
if map_size_data1 and map_size_data2:
    fig = side_by_side_multi_lines(
        epochs1, map_size_data1,
        epochs2, map_size_data2,
        y_label="mAP",
        title1=f"{label_1} - mAP by Size",
        title2=f"{label_2} - mAP by Size",
    )
    save_only(fig, "side_by_side_val_map_sizes", "Validation mAP by Object Size Comparison")

# 7) Average Recall curves
ar_metrics = ["AR@1", "AR@10", "AR@100"]
ar_data1 = {k: v for k, v in coco_metrics1.items() if k in ar_metrics}
ar_data2 = {k: v for k, v in coco_metrics2.items() if k in ar_metrics}
if ar_data1 and ar_data2:
    fig = side_by_side_multi_lines(
        epochs1, ar_data1,
        epochs2, ar_data2,
        y_label="AR",
        title1=f"{label_1} - Average Recall",
        title2=f"{label_2} - Average Recall",
    )
    save_only(fig, "side_by_side_val_ar", "Validation Average Recall Comparison")

# 8) AR by object size
ar_size_metrics = ["AR small", "AR medium", "AR large"]
ar_size_data1 = {k: v for k, v in coco_metrics1.items() if k in ar_size_metrics}
ar_size_data2 = {k: v for k, v in coco_metrics2.items() if k in ar_size_metrics}
if ar_size_data1 and ar_size_data2:
    fig = side_by_side_multi_lines(
        epochs1, ar_size_data1,
        epochs2, ar_size_data2,
        y_label="AR",
        title1=f"{label_1} - AR by Size",
        title2=f"{label_2} - AR by Size",
    )
    save_only(fig, "side_by_side_val_ar_sizes", "Validation AR by Object Size Comparison")

# 9) Individual mAP metrics
individual_metrics = ["mAP@[0.50:0.95]", "mAP@0.50", "mAP@0.75"]
for metric in individual_metrics:
    if metric in coco_metrics1 and metric in coco_metrics2:
        fig = side_by_side_single_line(
            epochs1, coco_metrics1[metric],
            epochs2, coco_metrics2[metric],
            y_label="mAP",
            title1=f"{label_1} - {metric}",
            title2=f"{label_2} - {metric}",
        )
        safe_name = metric.replace('@', '_').replace('[', '').replace(']', '').replace(':', '_')
        save_only(fig, f"side_by_side_{safe_name}", f"{metric} Comparison")

print(f"Done. HTML files saved in: {save_dir}")

Loading CSV files...
Creating Plotly side-by-side figures and saving HTML...
Saved: ./side_by_side_plots_plotly\side_by_side_train_loss.html
Saved: ./side_by_side_plots_plotly\side_by_side_train_loss_components.html
Saved: ./side_by_side_plots_plotly\side_by_side_aux_losses.html
Saved: ./side_by_side_plots_plotly\side_by_side_dn_losses.html
Saved: ./side_by_side_plots_plotly\side_by_side_val_map.html
Saved: ./side_by_side_plots_plotly\side_by_side_val_map_sizes.html
Saved: ./side_by_side_plots_plotly\side_by_side_val_ar.html
Saved: ./side_by_side_plots_plotly\side_by_side_val_ar_sizes.html
Saved: ./side_by_side_plots_plotly\side_by_side_mAP_0.50_0.95.html
Saved: ./side_by_side_plots_plotly\side_by_side_mAP_0.50.html
Saved: ./side_by_side_plots_plotly\side_by_side_mAP_0.75.html
Done. HTML files saved in: ./side_by_side_plots_plotly
